In [53]:
from ont_fast5_api.fast5_interface import get_fast5_file
import mappy

from pathlib import Path
from dataclasses import dataclass
import numpy as np
import sys
import struct

from typing import *

In [29]:
FASTQ_PATH = 'BaseCalled_template/Fastq'
MOVE_TABLE_PATH = 'BaseCalled_template/Move'

@dataclass
class ReadInfo:
    read_id: str
    fastq: str
    signal: np.ndarray
    move_table: np.ndarray
    block_stride: int

    def get_seq_to_sig(self) -> np.ndarray:
        move_table = np.append(self.move_table,
                               1)  # Adding for easier indexing
        return move_table.nonzero()[0] * self.block_stride

    def get_seq_and_quals(self) -> Tuple[str, np.ndarray]:
        data = self.fastq.strip().split('\n')

        sequence = data[1]
        quals = np.array([ord(c) - 33 for c in data[3]], dtype=np.uint8)

        return sequence, quals

    def get_normalized_signal(self, start=0, end=None) -> np.ndarray:
        signal = self.signal[start:end]
        med = np.median(signal)
        mad = np.median(np.abs(signal - med))

        return (signal - med) / (1.4826 * mad)


with get_fast5_file('single/0/0007dcce-b86b-4221-80ee-ea3ab41ad801.fast5', mode='r') as f5:
    read = list(f5.get_reads())[0]
        
    bc_analysis = read.get_latest_analysis('Basecall_1D')
    bc_summary = read.get_summary_data(bc_analysis)

    block_stride = bc_summary['basecall_1d_template']['block_stride']
    move_table = read.get_analysis_dataset(bc_analysis, MOVE_TABLE_PATH)
    fastq = read.get_analysis_dataset(bc_analysis, FASTQ_PATH)

    seg_analysis = read.get_latest_analysis('Segmentation')
    seg_summary = read.get_summary_data(seg_analysis)
    start = seg_summary['segmentation']['first_sample_template']
    signal = read.get_raw_data(start=start, scale=True)

    read_info = ReadInfo(read.read_id, fastq, signal, move_table, block_stride)

In [34]:
aligner = mappy.Aligner('chr20.fastq', best_n=1)

In [35]:
seq_to_sig = read_info.get_seq_to_sig()
signal = read_info.get_normalized_signal(end=seq_to_sig[-1]) \
                        .astype(np.half)
query, _ = read_info.get_seq_and_quals()

alignments = list(aligner.map(query))
len(alignments)

2

In [41]:
@dataclass
class AlignmentInfo:
    ctg: str
    r_start: int
    r_end: int
    fwd_strand: bool
    ref_to_query: np.ndarray32601808
    
alignment = alignments[0]

ref_len = alignment.r_en - alignment.r_st
cigar = alignment.cigar if alignment.strand == 1 else reversed(alignment.cigar)
rpos, qpos = 0, alignment.q_st # if alignment.strand == 1 else len(query) - alignment.q_en

ref_to_query = np.empty((ref_len + 1,), dtype=int)
for l, op in cigar:
    if op == 0 or op == 7 or op == 8:  # Match or mismatch
        for i in range(l):
            ref_to_query[rpos + i] = qpos + i
        rpos += l
        qpos += l
    elif op == 1:  # Insertion
        qpos += l
    elif op == 2:
        for i in range(l):
            ref_to_query[rpos + i] = qpos
        rpos += l
ref_to_query[rpos] = qpos  # Add the last one (excluded end)

aln_info = AlignmentInfo(alignment.ctg, alignment.r_st, alignment.r_en, 
                     alignment.strand == 1, ref_to_query)

In [45]:
ref_seq = aligner.seq('chr20', alignment.r_st, alignment.r_en)
len(ref_seq), len(aln_info.ref_to_query)

(2428, 2429)

In [50]:
f_idx = open('nije_bitno.rf.idx', 'rb')
n_examples = int.from_bytes(f_idx.read(4), byteorder=sys.byteorder)
print(n_examples)

indices = [int.from_bytes(f_idx.read(8), byteorder=sys.byteorder) for _ in range(n_examples)]

4681


In [54]:
data = open('nije_bitno.rf', 'rb')

all_examples = []
for _ in range(n_examples):
    data.seek(indices[0])
    r_id, ctg, pos, n_points = struct.unpack('=36sHIH', data.read(44))
    if r_id.decode() == '0007dcce-b86b-4221-80ee-ea3ab41ad801':
        *points, seq = struct.unpack(f'{n_points}e31s', data.read(n_points*2+31))
        all_examples.append((pos, points, seq.decode()))
        
len(all_examples)

4681

In [56]:
example = all_examples[0]
example[0]

32601808

In [57]:
rel_pos = 32601808 - alignment.r_st
ref_seq[rel_pos:rel_pos+2]

'CG'

In [58]:
aln_info.ref_to_query[rel_pos-15:rel_pos+16]

array([ 85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115])

In [60]:
seq_to_sig[85], seq_to_sig[115]
signal[710:940]

array([-0.9487 , -0.762  , -0.839  , -0.795  , -0.6743 , -0.4004 ,
       -0.3455 , -0.3674 , -0.3345 , -0.3015 , -0.3674 , -0.3345 ,
       -0.444  , -0.2249 , -0.159  ,  1.793  ,  1.946  ,  1.9795 ,
        1.925  ,  2.09   ,  1.497  ,  1.99   ,  1.256  ,  0.74   ,
        0.839  ,  0.9595 , -0.3674 , -0.3894 , -0.6523 , -0.6855 ,
       -0.5317 , -0.6523 , -0.499  , -0.5977 , -0.587  , -0.6636 ,
       -0.5977 , -0.6196 , -0.5537 , -0.6743 , -0.499  , -0.521  ,
       -0.2905 , -0.6743 , -0.609  , -0.1371 , -0.609  , -0.806  ,
       -0.5317 , -0.6636 , -0.74   , -0.3674 , -0.5537 , -0.444  ,
       -0.1042 , -0.3564 , -0.817  , -0.9487 , -1.091  , -1.321  ,
       -0.9595 , -1.179  , -0.5757 , -0.4004 , -0.2905 , -0.4004 ,
       -0.4004 , -0.3455 , -0.3674 , -0.444  ,  0.499  ,  2.012  ,
        1.902  ,  1.464  ,  2.145  ,  1.596  ,  1.804  ,  1.946  ,
        1.925  ,  1.859  ,  1.124  ,  0.828  ,  0.839  ,  0.7295 ,
        0.795  ,  0.6196 ,  0.5317 ,  0.521  , -0.2688 , -0.38

In [61]:
example[1]

[-0.94873046875,
 -0.76220703125,
 -0.8388671875,
 -0.794921875,
 -0.67431640625,
 -0.400390625,
 -0.345458984375,
 -0.367431640625,
 -0.33447265625,
 -0.301513671875,
 -0.367431640625,
 -0.33447265625,
 -0.444091796875,
 -0.224853515625,
 -0.1590576171875,
 1.79296875,
 1.9462890625,
 1.9794921875,
 1.9248046875,
 2.08984375,
 1.4970703125,
 1.990234375,
 1.255859375,
 0.740234375,
 0.8388671875,
 0.95947265625,
 -0.367431640625,
 -0.389404296875,
 -0.65234375,
 -0.685546875,
 -0.53173828125,
 -0.65234375,
 -0.4990234375,
 -0.59765625,
 -0.5869140625,
 -0.66357421875,
 -0.59765625,
 -0.61962890625,
 -0.5537109375,
 -0.67431640625,
 -0.4990234375,
 -0.52099609375,
 -0.29052734375,
 -0.67431640625,
 -0.60888671875,
 -0.1370849609375,
 -0.60888671875,
 -0.80615234375,
 -0.53173828125,
 -0.66357421875,
 -0.740234375,
 -0.367431640625,
 -0.5537109375,
 -0.444091796875,
 -0.10418701171875,
 -0.3564453125,
 -0.81689453125,
 -0.94873046875,
 -1.0908203125,
 -1.3212890625,
 -0.95947265625,
 -1

In [20]:
with get_fast5_file('single/0/0007dcce-b86b-4221-80ee-ea3ab41ad801.fast5', mode='r') as f5:
    read = list(f5.get_reads())[0]
    print(read.read_id)

0007dcce-b86b-4221-80ee-ea3ab41ad801
